<a href="https://colab.research.google.com/github/lwa01/229352-STAT-LEARING-FOR-DATA-SCI-2/blob/main/Lab03_kNN_using_Grid_Search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Statistical Learning for Data Science 2 (229352)
#### Instructor: Donlapark Ponnoprat

#### [Course website](https://donlapark.pages.dev/229352/)

## Lab #3

In [ ]:
from sklearn.datasets import fetch_20newsgroups

train = fetch_20newsgroups(subset='train')
test = fetch_20newsgroups(subset='test')

Xtrain = train.data[:3000]
ytrain = train.target[:3000]
Xtest = test.data[:500]
ytest = test.target[:500]

print("X:", len(Xtest))
print("y:", len(ytest))

In [ ]:
print("X[0]:", Xtrain[0])
print("y[0]:", ytrain[0])

In [ ]:
train.target_names

### Apply Tfidf ([documentation](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html))

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# define transformation
tfidf = TfidfVectorizer(stop_words = "english" , ngram_range = (1,1))

# fit+transform training set
Xtrain_tfidf = tfidf.fit_transform(Xtrain)
Xtest_tfidf = tfidf.transform(Xtest)

# See output
Xtrain_tfidf

In [ ]:
tfidf.vocabulary_

#### Exercise 1: Find post in the training set that is closest in tf-idf to the first post in the test set (`Xtest[0]`). Print the content of both posts (not the tf-idf vectors).

In [ ]:
test_vec = Xtest_tfidf[0]

min_dist = float("inf")
best_index = - 1

for i in range(Xtrain_tfidf.shape[0]):
    train_vec = Xtrain_tfidf[i]

    diff = test_vec - train_vec
    dist = np.sqrt(diff.multiply(diff).sum())

    if dist < min_dist:
        min_dist = dist
        best_index = i

In [ ]:
print(best_index)
print(min_dist)

print(Xtest[0])

print(Xtrain[best_index])

### Classify with k-Nearest Neighbor (kNN) ([documentation](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html))

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

# Define model
nb = KNeighborsClassifier(n_neighbors = 3)

# Fit the model
nb.fit(Xtrain_tfidf, ytrain)

Evaluate on the test set using [`classification_report`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html)

We will focus on the [F1-score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html)

In [ ]:
from sklearn.metrics import classification_report
import numpy as np

# Transform the test set
Xtest_tfidf = tfidf.transform(Xtest)

# Make predictions on the test set
ypred = nb.predict(Xtest_tfidf)
np.array(train.target_names)[ypred]

# Report classification scores
print(classification_report(ytest, ypred))

### Combine all methods into a [pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html)

In [ ]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([('tfidf', TfidfVectorizer()),('nb', KNeighborsClassifier())])

# Fit the pipeline to the training set
pipeline.fit(Xtrain, ytrain)

# Make predictions on the test set
ypred = pipeline.predict(Xtest)

# report classification scores
print(classification_report(ytest, ypred))

Now we will use [grid search cross-validation](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) to find model with the best hyperparameters

![5CV](https://scikit-learn.org/stable/_images/grid_search_cross_validation.png)

In [ ]:
from sklearn.model_selection import GridSearchCV

params = {"tfidf__ngram_range": [(1,1), (1,2)],
          "nb__n_neighbors": [1,3,5]}

# Define GridSearchCV
gridcv = GridSearchCV(pipeline, params,
                      scoring='f1_macro', cv = 3)

# Fit and cross-validate the model on 3-fold data
gridcv.fit(Xtrain, ytrain)


In [ ]:
gridcv.best_estimator_

In [ ]:
# Make predictions on the test set
ypred = gridcv.predict(Xtest)

# Report classification scores
print(classification_report(ytest, ypred))

#### Exercise 2:

1. Use grid search 5-fold cross-validation across different values of the following two kNN parameters: `n_neighbors` and `metric`  **on the training set** to find the best model.

2. For the best value of `n_neighbors` and `metric` you found above, compute the `f1_macro` score **on the test set**.
* Print the value of `n_neighbors` and `metric`.
* Print the model's `f1_macro` score.

In [ ]:
# 1.
from sklearn.model_selection import GridSearchCV

params = {"nb__n_neighbors": [1,3,5,7,9],
          "nb__metric": ["euclidean", "manhattan", "minkowski"]}

# Define GridSearchCV
gridcv = GridSearchCV(pipeline, params,
                      scoring='f1_macro', cv = 5)

# Fit and cross-validate the model on 3-fold data
gridcv.fit(Xtrain, ytrain)


In [ ]:
# 2.
best_n_neighbors = gridcv.best_params_["nb__n_neighbors"]
best_metric = gridcv.best_params_["nb__metric"]

print("Best n_neighbors:", best_n_neighbors)
print("Best metric:", best_metric)

In [ ]:
best_model = gridcv.best_estimator_

In [ ]:
from sklearn.metrics import f1_score
ypred = best_model.predict(Xtest)

f1_macro = f1_score(ytest, ypred, average="macro")
print("Test f1_macro: ", f1_macro)

# Report classification scores
print(classification_report(ytest, ypred))
